## Selenium

### Chrome chromedriver
https://chromedriver.storage.googleapis.com/index.html?path=2.28/
Unzip downlaoded chromedriver.zip
sudo mv ~/Downloads/chromedriver /usr/local/bin/

### Firefox geckodriver
https://github.com/mozilla/geckodriver/releases
tar xzvf gekodriver-v0.19.1-linux64.tar.gz
sudo mv ~/Downloads/geckodriver /usr/local/bin/

In [ ]:
!pip install splinter

In [ ]:
#refer to https://python-forum.io/Thread-facebook-friends-crawler
#refer to http://blog.munhou.com/facebook-peng-you-da-tou-tie-pa-chong/
#Have to add time sleep on click and scrolling for human behavior simulation

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import selenium.common.exceptions
import requests
import getpass
import time
import os
import shutil
import re
import json

CLICK_PAUSE_TIME = 0.2
SCROLL_PAUSE_TIME = 1

username = input('Username:')
password = getpass.getpass('Password:')

retries = 3
timeout = 10

browser = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')
# coptions = webdriver.ChromeOptions()
# coptions.add_argument('--no-sandbox')
# coptions.add_argument('--headless')
# coptions.add_argument('--disable-gpu')
# coptions.add_argument('--disable-infobars')
# browser = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver', chrome_options=coptions)

browser.implicitly_wait(timeout)

profile_pages = []
friends = []
cookies = {}
photos_dir= 'photos'
if not os.path.exists(photos_dir):
    os.makedirs(photos_dir)


def facebook():
    # login fb
    browser.get('https://www.facebook.com')
    time.sleep(CLICK_PAUSE_TIME)
    assert 'Facebook' in browser.title
    
    #send username
    elem = browser.find_element_by_name('email')
    elem.send_keys(username)
    
    #send password
    elem = browser.find_element_by_name('pass')
    elem.send_keys(password)
    
    #click login
    elem = browser.find_element_by_id('loginbutton')
    time.sleep(CLICK_PAUSE_TIME)
    elem.click()
    time.sleep(CLICK_PAUSE_TIME)
    return

def get_friend_list():  
     # go to profile page
    elem = browser.find_element_by_xpath('//a[@title="個人檔案"]')
    #elem = browser.find_element_by_xpath('//a[@title="Profile"]')
    time.sleep(CLICK_PAUSE_TIME)
    elem.click()
    time.sleep(CLICK_PAUSE_TIME)

    # go to friend page
    elem = browser.find_element_by_xpath('//a[@data-tab-key="friends"]')
    time.sleep(CLICK_PAUSE_TIME)
    elem.click()
    time.sleep(CLICK_PAUSE_TIME)

    # click on all friends tab
    elem = browser.find_element_by_xpath('//a[@name="所有朋友"]')
    #elem = browser.find_element_by_xpath('//a[@name="All friends"]')
    link = elem.get_attribute('href')
    browser.get(link)

    # scroll to bottom
    scroll_to_bottom()

    # get all friend profile page url
    elems = browser.find_elements_by_xpath('//div[@class="uiProfileBlockContent"]/div/div/div/a')
    total_count = 0
    for elem in elems:
        if total_count%10 == 0:
            profile_pages.append([])
        profile_pages[total_count//10].append(elem.get_attribute('href'))
        total_count += 1
    print(total_count)
    return 

def download_profile_picture():
    print(profile_pages)
    print("\n")
    
    i=0
    # get all friend profile pictures
    for page_list in profile_pages:
        for page in page_list:
            browser.get(page)
            friends.append({})
              
            # get friend's name and id
            elem = browser.find_element_by_id('fb-timeline-cover-name')
            friends[i]['name'] = elem.text.replace('\n', '')
            friends[i]['id'] = re.findall('"entity_id":"([0-9]+)"', browser.page_source)[0]
            friends[i]['fburl'] = page
            
            print("%d %s id:%s"%(i+1,friends[i]['name'],friends[i]['id']))

            # get cookie
            all_cookies = browser.get_cookies()
            for s_cookie in all_cookies:
                cookies[s_cookie["name"]] = s_cookie["value"]

            # get image link
            link = get_image_link()
            friends[i]['imglink'] = link
            print("link: %s "%(friends[i]['imglink']))

            # download image
            if link:
                response = requests.get(link, stream=True, cookies=cookies)              
                if re.findall("(\d+_\S+.jpg)", link) == []:
                    filename = re.findall("filename=(\S+)", response.headers['Content-Disposition'])[0]
                else:
                    filename = re.findall("(\d+_\S+.jpg)", link)[0]
                friends[i]['imgname'] = filename
                print("filename: %s"%(friends[i]['imgname']))
                with open(os.path.join(photos_dir, filename), 'wb') as out_file:
                    response.raw.decode_content = True
                    shutil.copyfileobj(response.raw, out_file)
                del response
            else:
                print('skip %s' % friends[i]['name'])
            cookies.clear()
            i += 1
            
                       
    return

def get_image_link():
    link = None
    n_retry = 0
    while not link and n_retry < retries:
        try:
            # click on thumbnail
            elem = browser.find_element_by_xpath('//a[@class="profilePicThumb"]')
            time.sleep(CLICK_PAUSE_TIME)
            elem.click()
            time.sleep(CLICK_PAUSE_TIME)

            # mouseover to the profile picture
            WebDriverWait(browser, timeout//5).until(EC.presence_of_element_located((By.ID, 'photos_snowlift')))
            hover = ActionChains(browser).move_to_element(elem)
            hover.perform()

            # click on option button
            elem = browser.find_element_by_xpath('//a[@data-action-type="open_options_flyout"]')
            time.sleep(CLICK_PAUSE_TIME)
            elem.click()
            time.sleep(CLICK_PAUSE_TIME)

            # search for downloadable photo
            elem = browser.find_element_by_xpath('//*[@data-action-type="download_photo"]/a')
            link = elem.get_attribute('href')

        # for not a downloadable photo
        except selenium.common.exceptions.TimeoutException:
            try:
                elem = browser.find_element_by_xpath(
                    '//div[@role="presentation"]/div[@role="presentation"]/div/div/div/img')
                link = elem.get_attribute('src')

            # try again
            except selenium.common.exceptions.NoSuchElementException:
                print('try again')
                n_retry += 1
                continue

        # try again
        except selenium.common.exceptions.NoSuchElementException:
            print('try again')
            n_retry += 1
            continue

    return link

def scroll_to_bottom():
    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")
 
    while True:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
 
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
 
        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
facebook()
get_friend_list()
download_profile_picture()
with open('fb_friends.json', 'w', encoding='utf8') as fp:
    json.dump(friends, fp, ensure_ascii=False)